In [1]:
from pymongo import MongoClient
import pandas as pd
import time

In [2]:
client = MongoClient("localhost:27017")

In [3]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
client.list_database_names()

['IronHack', 'admin', 'config', 'local']

In [5]:
db = client["IronHack"]

In [6]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'IronHack')

In [7]:
c = db.get_collection('companies')

In [8]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [9]:
filter_ = {"category_code":"games_video"}
projection = {"_id": 0, "name": 1, "offices.latitude": 1, "offices.longitude": 1, "offices.city": 1}

In [10]:
c.find_one(filter_, projection)

{'name': 'Flektor',
 'offices': [{'city': 'Culver City',
   'latitude': 34.025958,
   'longitude': -118.379768}]}

In [11]:
result = list(c.find(filter_, projection))

result[0]


{'name': 'Flektor',
 'offices': [{'city': 'Culver City',
   'latitude': 34.025958,
   'longitude': -118.379768}]}

In [12]:
name_list = []
long_list = []
lat_list = []
city_list = []

for i in range(0, len(result) - 1):
    if "offices" in result[i] and len(result[i]["offices"]) > 0:
        if "latitude" in result[i]["offices"][0] and "longitude" in result[i]["offices"][0]:
            name_list.append(result[i]["name"])
            lat_list.append(result[i]["offices"][0]["latitude"])
            long_list.append(result[i]["offices"][0]["longitude"])
            city_list.append(result[i]["offices"][0]["city"])


In [13]:
len(name_list)

838

In [14]:
len(lat_list)

838

In [15]:
len(long_list)

838

In [16]:
len(city_list)

838

In [17]:
# Creating dataframe
gaming_offices = pd.DataFrame(
    {'Name': name_list,
     'Latitude': long_list,
     'Longitude': lat_list,
     'City': city_list
    })

In [18]:
#Dropping rows with Nan values
gaming_offices.dropna(inplace=True)
gaming_offices

,Name,Latitude,Longitude,City
0,Flektor,-118.379768,34.025958,Culver City
1,Lala,-122.154369,37.451151,Palo Alto
2,Joost,-74.009447,40.746497,New York
3,Babelgum,-6.267494,53.344104,London
5,Kyte,-122.409173,37.788482,San Francisco
...,...,...,...,...
831,MechaWorks,-122.009405,37.383444,Sunnyvale
832,DJ Nitrogen,-122.449408,37.784174,San Francisco
833,Shape Games,-77.042423,38.814208,Alexandria
836,Fictionaut,-73.916127,40.759795,Astoria


In [19]:
# Finding most commonly occuring cities
gaming_offices["City"].value_counts()[:20]

New York         48
San Francisco    47
Los Angeles      24
                 23
London           18
Palo Alto        16
Seattle          13
Paris            11
Santa Monica      8
San Mateo         8
Vancouver         7
Sunnyvale         7
Miami             6
Mountain View     6
Chicago           6
Munich            5
Toronto           5
Boston            5
Redwood City      5
Cambridge         5
Name: City, dtype: int64

In [20]:
# 3 most commonly occuring cities are: New York, San Francisco, Los Angeles 

In [21]:
gaming_offices.columns

Index(['Name', 'Latitude', 'Longitude', 'City'], dtype='object')

In [22]:
cities_to_keep = ['New York', 'San Francisco', 'Los Angeles']

# drop rows where the value in column_1 is not in values_to_keep
gaming_offices = gaming_offices[gaming_offices['City'].isin(cities_to_keep)]
gaming_offices

,Name,Latitude,Longitude,City
2,Joost,-74.009447,40.746497,New York
5,Kyte,-122.409173,37.788482,San Francisco
8,Stickam,-118.254558,34.051409,Los Angeles
10,Livestream,-73.995625,40.726155,New York
11,Ustream,-122.079480,37.392936,San Francisco
...,...,...,...,...
809,Exent,-74.005568,40.752380,New York
811,Mondo Media,-122.402021,37.764169,San Francisco
826,Film Annex,-74.009447,40.746497,New York
827,FremantleMedia Ventures,-73.985542,40.743945,New York


In [23]:
gaming_offices['Latitude'], gaming_offices['Longitude'] = gaming_offices['Longitude'], gaming_offices['Latitude']
gaming_offices

/var/folders/6b/73zmt1sx3c30hcgymp7n2zw40000gn/T/ipykernel_21116/3099485257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gaming_offices['Latitude'], gaming_offices['Longitude'] = gaming_offices['Longitude'], gaming_offices['Latitude']


,Name,Latitude,Longitude,City
2,Joost,40.746497,-74.009447,New York
5,Kyte,37.788482,-122.409173,San Francisco
8,Stickam,34.051409,-118.254558,Los Angeles
10,Livestream,40.726155,-73.995625,New York
11,Ustream,37.392936,-122.079480,San Francisco
...,...,...,...,...
809,Exent,40.752380,-74.005568,New York
811,Mondo Media,37.764169,-122.402021,San Francisco
826,Film Annex,40.746497,-74.009447,New York
827,FremantleMedia Ventures,40.743945,-73.985542,New York


In [25]:
gaming_offices = gaming_offices.sort_values(by='City').reset_index(drop=True)
gaming_offices

,Name,Latitude,Longitude,City
0,Mybandstock,42.280268,-83.742058,Los Angeles
1,Taltopia,34.052187,-118.243425,Los Angeles
2,Social Gaming Network,37.446823,-122.161523,Los Angeles
3,Modern Feed,34.085749,-118.343791,Los Angeles
4,OVGuide,34.052187,-118.243425,Los Angeles
...,...,...,...,...
114,Serious Business,37.789321,-122.401362,San Francisco
115,Sutros,37.756922,-122.420870,San Francisco
116,stickychicken,37.775196,-122.419204,San Francisco
117,One Season,37.775196,-122.419204,San Francisco


In [26]:

import pandas as pd
import requests

def get_nearby_venues(lat, lng, category_code):
    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lng}&categories={category_code}"

    headers = {
        "accept": "application/json",
        "Authorization": "fsq3kP6YLxCVarFG20O8HCsrdOH1BPtM49DNvjZ0rmBFAbY="
    }

    response = requests.get(url, headers=headers).json()
    
    if response["results"]:
        name = response["results"][0]["name"]
        distance = response["results"][0]["distance"]
        return name, distance
    else:
        return None, None

category_codes = [
    "International Airport", 19040,
    "Train station", 19047,
    "Nightclub", 10032,
    "Pet grooming", 11134,
    "Vegan restaurant", 13377,
    "Basketball stadium", 18006,
    "Design studio", 11030,
    "Daycare", 11026,
    "Preschool", 12056,
    "High school", 12059
]

offices_df = gaming_offices[["Name", "Latitude", "Longitude"]].copy()

for i in range(0, len(category_codes), 2):
    category_name = category_codes[i]
    category_code = category_codes[i+1]
    column_name = f"{category_name}: Name"
    column_distance = f"{category_name}: Distance"
    offices_df[column_name] = None
    offices_df[column_distance] = None
    
    for index, row in offices_df.iterrows():
        lat = row["Latitude"]
        lng = row["Longitude"]
        name, distance = get_nearby_venues(lat, lng, category_code)
        offices_df.at[index, column_name] = name
        offices_df.at[index, column_distance] = distance

offices_df.head()


,Name,Latitude,Longitude,International Airport: Name,International Airport: Distance,Train station: Name,Train station: Distance,Nightclub: Name,Nightclub: Distance,Pet grooming: Name,...,Basketball stadium: Name,Basketball stadium: Distance,Design studio: Name,Design studio: Distance,Daycare: Name,Daycare: Distance,Preschool: Name,Preschool: Distance,High school: Name,High school: Distance
0,Mybandstock,42.280268,-83.742058,None,None,Ann Arbor Station,837,Nightcap,544,BreedAbove,...,Player Development Center,2320,Signs By Tomorrow - Ann Arbor,5750,Little Angels Preschool & Daycare,9979,Fumcn,117,Canton High School,21852
1,Taltopia,34.052187,-118.243425,Los Angeles International Airport (LAX),19233,Angels Flight - Upper Station,737,Monty Bar,2141,Tailwaggers,...,Champions,14037,Silver Screening Room - Pacific Design Center,13233,Murchison Early Education Center,3462,Pinocchio Child Care Center,4977,Palisades Charter High School,26503
2,Social Gaming Network,37.446823,-122.161523,San Francisco International Airport (SFO),27413,Mountain View Caltrain Station,9541,The Ritz,27588,Wag Hotels Redwood City,...,Sports House Indoor Sports and Cafe,4915,Facebook Analog Research Laboratory,4275,Love N Care Christian Preschool & Daycare,2846,New Beginnings Preschool,1955,Burlingame High School,22192
3,Modern Feed,34.085749,-118.343791,Los Angeles International Airport (LAX),16797,Angels Flight - Upper Station,9276,The Abbey Food & Bar,3824,Tailwaggers,...,Champions,13474,Silver Screening Room - Pacific Design Center,3511,A Kid's Place,12214,Pinocchio Child Care Center,4994,Palisades Charter High School,17776
4,OVGuide,34.052187,-118.243425,Los Angeles International Airport (LAX),19233,Angels Flight - Upper Station,737,Monty Bar,2141,Tailwaggers,...,Champions,14037,Silver Screening Room - Pacific Design Center,13233,Murchison Early Education Center,3462,Pinocchio Child Care Center,4977,Palisades Charter High School,26503


In [36]:
offices_df.to_csv(r'/Users/tom/desktop\officesproject.csv')